In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [4]:
def add_shifts(data, n_in, n_out):
    df = pd.DataFrame(data)
    cols, names = [], []
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names.append('t-{}'.format(i))
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        names.append('t' if i==0 else 't+{}'.format(i))
    result = pd.concat(cols,axis=1)
    result.columns = names
    result = result.dropna()
    return result

In [5]:
def prepare(data, n_test, n_in, n_out):
    arr = np.array(data.diff()).reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(-1,1))
    arr = add_shifts(scaler.fit_transform(arr), n_in, n_out).values
    return scaler, arr[:-n_test], arr[-n_test:]

In [6]:
def create_lstm(data, n_in, n_batch, n_epochs, n_neurons):
    X, y = data[:,:n_in], data[:,n_in:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(n_epochs):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [7]:
def next_forecast(model, X, n_batch):
    X = X.reshape(1,1,-1)
    return [x for x in model.predict(X, batch_size=n_batch)]

In [8]:
def predict(model, test, n_in, n_out, n_batch):
    forecasts = []
    for i in range(0,len(test),n_out):
        X = test[i,:n_in]
        forecasts.append(next_forecast(model, X, n_batch))
    return forecasts

In [9]:
def undiff(last, diffs):
    result = []
    result.append(last + diffs[0])
    for i in range(1, len(diffs)):
        result.append(diffs[i] + result[i-1])
    return result

In [10]:
def inverse_transform(data, forecasts, scaler, n_test):
    converted = []
    for i in range(len(forecasts)):
        forecast = scaler.inverse_transform(np.array(forecasts[i]))[0,:]
        index = len(data) - n_test + i - 1
        last = data[index]
        converted.append(undiff(last, forecast))
    return converted

In [11]:
def plot_forecast(data, forecasts, n_in, n_out):
    plt.figure(figsize=(20,10))
    plt.grid()
    plt.plot(data)
    for i in range(0,len(forecasts)):
        s = n_in + n_out*i - 1
        e = s + len(forecasts[i]) + 1
        x = [x for x in range(s, e)]
        y = [data[s]] + forecasts[i]
        plt.plot(x, y, color='red')
    plt.show()